# Part 6 Creation of a statistically representative snowpack using all SMP profiles around a surveyed snowpit
[Benoit Montpetit](https://github.com/ecccben), *CPS/CRD/ECCC*, 2024  
[Mike Brady](https://github.com/m9brady), *CPS/CRD/ECCC*, 2024

Here we use all SMP profiles measured for a given snowpit and compute the median values of all the snow geophysical properties for the two main grain type layers (rounded grains and depth hoar grains). This is used in the discussion section of [Montpetit et al. (2024)](https://doi.org/10.5194/egusphere-2024-651) to show that using a two layer snowpack does not have a significant impact on the simulated backscatter at Ku-Band using SMRT.

In [ ]:
import pandas as pd
import geopandas as gp
import numpy as np

from matplotlib import pyplot as plt
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

In [ ]:
df_aggpits = pd.read_json('../Data/df_aggpits.json')

In [ ]:
from constants import TVC02
sites = pd.DataFrame({'site':TVC02})
sites.site.replace({'RS':'RP'}, regex=True, inplace=True)
sites=list(sites.site.values)
sites.remove('SC02')

In [ ]:
mp = gp.read_file('../Data/ref_mp.geojson')

In [ ]:
pd.DataFrame(df_aggpits[site][profile])

In [ ]:
pit_stats = {}

rho_ws = np.array([])
rho_dh = np.array([])
ssa_ws = np.array([])
ssa_dh = np.array([])
pex_ws = np.array([])
pex_dh = np.array([])
t_ws = np.array([])
t_dh = np.array([])
dh_frac = np.array([])

for site in sites:

    pit_stats[site]={}

    for profile in df_aggpits[site].dropna().keys():

        pit = pd.DataFrame(df_aggpits[site][profile])

        rho_ws=np.concatenate((rho_ws,pit['density'][pit['grain_type']=='R']))
        rho_dh=np.concatenate((rho_dh,pit['density'][pit['grain_type']=='H']))

        ssa_ws=np.concatenate((ssa_ws,pit['ssa'][pit['grain_type']=='R']))
        ssa_dh=np.concatenate((ssa_dh,pit['ssa'][pit['grain_type']=='H']))

        pex_ws=np.concatenate((pex_ws,pit['p_ex'][pit['grain_type']=='R']))
        pex_dh=np.concatenate((pex_dh,pit['p_ex'][pit['grain_type']=='H']))

        t_ws=np.concatenate((t_ws,pit['temperature'][pit['grain_type']=='R']))
        t_dh=np.concatenate((t_dh,pit['temperature'][pit['grain_type']=='H']))
        
        dh_frac=np.concatenate((dh_frac,[pit['thickness'][pit['grain_type']=='H'].sum()/
                                pit['thickness'].sum()]))

    depth = mp.loc[(mp.site==site.replace('RP','RS'))].depth.median()

    pit_stats[site]={'h':depth, 'dh_frac':np.median(dh_frac),
                     'rho_ws':np.median(rho_ws),'rho_dh':np.median(rho_dh),
                     'ssa_ws':np.median(ssa_ws),'ssa_dh':np.median(ssa_dh),
                     'pex_ws':np.median(pex_ws),'pex_dh':np.median(pex_dh),
                     't_ws':np.median(t_ws),'t_dh':np.median(t_dh)}
    
stat_pits = pd.DataFrame(pit_stats)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,10))

hist_ws = ax[0].hist2d(rho_ws, ssa_ws,bins=50,density=True, vmax = 0.001, range = [[100,500], [0,70]])
hist_dh = ax[1].hist2d(rho_dh, ssa_dh,bins=50,density=True, vmax = 0.001, range = [[100,500], [0,70]])

gt = ['Rounded Grains', 'Depth Hoar']

for i in range(2):

    ax[i].set_xlabel('$\\rho_{snow}$ $(kg \\cdot m^3)$')
    ax[i].set_ylabel('SSA $m^2 \\cdot kg^{-1}$')
    ax[i].set_title(gt[i])
    ax[i].set_ylim(0,70)
    ax[i].set_xlim(100,500)



<center><img src="../Figures/Part_6_StatPit_Fig1.png" Width="1000px"></center>

<center>Figure: Heat map of SSA and density for rounded grains and depth hoar grains.</center>

In [ ]:
stat_pits.to_json('../Data/df_Site_pit_stats.json')

In [ ]:
pits={}

for site in sites:

    pits[site]={}

    pits[site]['thickness']=np.array([stat_pits[site].h*(1-stat_pits[site].dh_frac), stat_pits[site].h*stat_pits[site].dh_frac])/100
    pits[site]['density']=np.array([stat_pits[site].rho_ws,stat_pits[site].rho_dh])
    pits[site]['ssa']=np.array([stat_pits[site].ssa_ws,stat_pits[site].ssa_dh])
    pits[site]['temperature']=np.array([stat_pits[site].t_ws,stat_pits[site].t_dh])
    pits[site]['p_ex']=np.array([stat_pits[site].pex_ws,stat_pits[site].pex_dh])
    pits[site]['grain_type']=np.array(['R','H'])

df_pits = pd.DataFrame(pits)

In [ ]:
df_pits

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>RP16</th>
      <th>RP17</th>
      <th>RP18</th>
      <th>RP19</th>
      <th>RP20</th>
      <th>RP21</th>
      <th>RP22</th>
      <th>RP23</th>
      <th>RP24</th>
      <th>RP25</th>
      <th>...</th>
      <th>RP27</th>
      <th>RP28</th>
      <th>RP29</th>
      <th>RP30</th>
      <th>RP31</th>
      <th>SD02</th>
      <th>SM02</th>
      <th>SO02</th>
      <th>ST02</th>
      <th>SV02</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>thickness</th>
      <td>[0.18558919469928653, 0.3178108053007135]</td>
      <td>[0.17724921756237141, 0.2913507824376286]</td>
      <td>[0.17405298220513277, 0.28609701779486724]</td>
      <td>[0.16158420684835784, 0.2706157931516422]</td>
      <td>[0.16638596491228072, 0.3078140350877193]</td>
      <td>[0.15386206896551718, 0.2923379310344828]</td>
      <td>[0.16799999999999993, 0.31920000000000004]</td>
      <td>[0.16062068965517237, 0.3051793103448276]</td>
      <td>[0.17453607932875667, 0.3250639206712433]</td>
      <td>[0.16449122807017547, 0.3043087719298246]</td>
      <td>...</td>
      <td>[0.1680346820809248, 0.31646531791907523]</td>
      <td>[0.16073684210526318, 0.2973631578947369]</td>
      <td>[0.1634914285714286, 0.31335857142857143]</td>
      <td>[0.16605780346820803, 0.3127421965317919]</td>
      <td>[0.1373859649122807, 0.25416403508771934]</td>
      <td>[0.16484210526315793, 0.30495789473684204]</td>
      <td>[0.15536094674556208, 0.2822390532544379]</td>
      <td>[0.136, 0.24479999999999996]</td>
      <td>[0.1600714285714286, 0.2881285714285714]</td>
      <td>[0.11479279279279282, 0.2037572072072072]</td>
    </tr>
    <tr>
      <th>density</th>
      <td>[278.6, 228.0]</td>
      <td>[318.35, 227.0]</td>
      <td>[321.0, 227.2]</td>
      <td>[331.0, 227.7]</td>
      <td>[333.65, 228.5]</td>
      <td>[329.6, 228.5]</td>
      <td>[323.45, 228.8]</td>
      <td>[316.0, 228.8]</td>
      <td>[316.35, 228.7]</td>
      <td>[317.15, 228.8]</td>
      <td>...</td>
      <td>[323.45, 228.8]</td>
      <td>[327.5, 228.8]</td>
      <td>[324.4, 228.9]</td>
      <td>[329.6, 229.0]</td>
      <td>[331.4, 229.1]</td>
      <td>[334.25, 229.1]</td>
      <td>[334.6, 229.2]</td>
      <td>[334.25, 229.2]</td>
      <td>[336.3, 229.125]</td>
      <td>[334.0, 229.2]</td>
    </tr>
    <tr>
      <th>ssa</th>
      <td>[32.237440026211466, 13.260924180861092]</td>
      <td>[33.44060878312321, 13.173104169866324]</td>
      <td>[33.035374154514294, 13.159402693095366]</td>
      <td>[32.990097573209106, 12.81056364989967]</td>
      <td>[32.6960731441793, 12.677143284704822]</td>
      <td>[32.21072877865501, 12.676163934624377]</td>
      <td>[32.25207636997817, 12.581449794616653]</td>
      <td>[31.529470602003602, 12.622513523865415]</td>
      <td>[31.468707355348386, 12.669502248059436]</td>
      <td>[31.337698045622908, 12.584106063545248]</td>
      <td>...</td>
      <td>[31.27415130950995, 12.499272347995621]</td>
      <td>[31.24416917482302, 12.489180956550133]</td>
      <td>[31.21497944806069, 12.473856988475742]</td>
      <td>[31.475627982876855, 12.49038930981996]</td>
      <td>[31.529470602003602, 12.440239698418361]</td>
      <td>[31.59304042253016, 12.440239698418361]</td>
      <td>[31.77860228312096, 12.423361959486721]</td>
      <td>[31.698936998180635, 12.453873170254756]</td>
      <td>[31.936778694604413, 12.428395825161711]</td>
      <td>[31.698936998180635, 12.418933233592092]</td>
    </tr>
    <tr>
      <th>temperature</th>
      <td>[243.4499969482422, 254.84999084472656]</td>
      <td>[245.14999389648438, 254.64999389648438]</td>
      <td>[243.4499969482422, 253.25]</td>
      <td>[243.4499969482422, 253.04998779296875]</td>
      <td>[243.4499969482422, 252.25]</td>
      <td>[243.14999389648438, 252.25]</td>
      <td>[243.14999389648438, 252.4499969482422]</td>
      <td>[243.4499969482422, 252.75]</td>
      <td>[243.4499969482422, 252.84999084472656]</td>
      <td>[243.75, 253.04998779296875]</td>
      <td>...</td>
      <td>[243.75, 253.04998779296875]</td>
      <td>[244.14999389648438, 253.04998779296875]</td>
      <td>[244.34999084472656, 254.4499969482422]</td>
      <td>[245.14999389648438, 254.4499969482422]</td>
      <td>[245.54998779296875, 253.25]</td>
      <td>[246.04998779296875, 253.25]</td>
      <td>[246.04998779296875, 253.9499969482422]</td>
      <td>[245.64999389648438, 253.54998779296875]</td>
      <td>[245.54998779296875, 253.54998779296875]</td>
      <td>[245.39999389648438, 253.25]</td>
    </tr>
    <tr>
      <th>p_ex</th>
      <td>[8.708015451643231e-05, 0.0002473461491094519]</td>
      <td>[7.84462710520973e-05, 0.0002480702136766012]</td>
      <td>[7.865317227584597e-05, 0.0002492198832124209]</td>
      <td>[7.865611192792172e-05, 0.00025649296561298485]</td>
      <td>[7.899550930006376e-05, 0.0002588801306794322]</td>
      <td>[8.094302362066977e-05, 0.0002577358748423371]</td>
      <td>[8.331209584236973e-05, 0.0002601671929891305]</td>
      <td>[8.713510268362186e-05, 0.000259964642973183]</td>
      <td>[8.710762860002708e-05, 0.00025809948328581753]</td>
      <td>[8.68937426768748e-05, 0.0002603561066004128]</td>
      <td>...</td>
      <td>[8.6631408504851e-05, 0.00026123755823841615]</td>
      <td>[8.623756536916503e-05, 0.00026123755823841615]</td>
      <td>[8.655236759517149e-05, 0.00026131401697781706]</td>
      <td>[8.582890156261045e-05, 0.00026110354174397384]</td>
      <td>[8.485575975364736e-05, 0.00026176267612973983]</td>
      <td>[8.546529614847099e-05, 0.00026176267612973983]</td>
      <td>[8.55279453335924e-05, 0.0002620704174646179]</td>
      <td>[8.570758500295788e-05, 0.000261390475717218]</td>
      <td>[8.44545991201085e-05, 0.00026207672933773217]</td>
      <td>[8.55279453335924e-05, 0.0002621434893325947]</td>
    </tr>
    <tr>
      <th>grain_type</th>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>...</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
      <td>[R, H]</td>
    </tr>
  </tbody>
</table>
<p>6 rows × 21 columns</p>
</div>

In [ ]:
df_pits.to_json('../Data/df_stat_pits.json')